# Module 6: Linear Support Vector Machine - Practice

In this session, you will practice using Linear SVM on **red wine** dataset
with the typical train/validate workflow.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os, sys
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

tf.logging.set_verbosity(tf.logging.ERROR)

## Load dataset

In the following cell, **print out class distribution** before and after labels are binarized respectively.

In [2]:
# Dataset location
DATASET = '/dsa/data/all_datasets/wine-quality/winequality-red.csv'
assert os.path.exists(DATASET)

# Load and shuffle
dataset = pd.read_csv(DATASET, sep=';').sample(frac = 1).reset_index(drop=True)

In [3]:
# Pull features and labels
selected_features = [1,6,9,10]
X = scale(np.array(dataset.iloc[:, selected_features]))
y = np.array(dataset.quality)

# Complete code below this comment  (Question #P6001)
# ----------------------------------
print('Class distribution (before binarization):', {i:np.sum(y==i) for i in np.unique(y)})

# Binarize labels
y = y>=6
print('Class distribution (after binarization):', {i: np.sum(y==i) for i in np.unique(y)})
# ----------------------------------

# Create training/validation split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

dataset.describe()

Class distribution (before binarization): {3: 10, 4: 53, 5: 681, 6: 638, 7: 199, 8: 18}
Class distribution (after binarization): {False: 744, True: 855}


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


## Create linear SVM model

Collect names for all selected feature columns.

In [4]:
feature_names = [dataset.columns[i].replace('\x20', '_') for i in selected_features]
print(feature_names)

['volatile_acidity', 'total_sulfur_dioxide', 'sulphates', 'alcohol']


Create feature columns placeholders for TensorFlow SVM.

In [5]:
# Complete code below this comment  (Question #P6002)
# ----------------------------------
feature_columns = [tf.contrib.layers.real_valued_column(i) for i in feature_names]

Create a linear classifier.

In [6]:
# Complete code below this comment  (Question #P6003)
# ----------------------------------
classifier = tf.contrib.learn.SVM('example_id', feature_columns=feature_columns, l2_regularization=1.0)

## Training and preparation

Create input_fn() to supply training data for linear SVM.

In [7]:
# Complete code below this comment  (Question #P6004)
# ----------------------------------
def my_input_fn():
    columns = {
        feature_name: tf.constant(np.expand_dims(X_train[:,i], 1))
            for i,feature_name in enumerate(feature_names)
    }
    columns['example_id'] = tf.constant([str(i+1) for i in range(len(X_train))])
    labels = tf.constant(y_train)
    return columns, labels


Train SVM.

In [8]:
# Add code below this comment  (Question #P6005)
# ----------------------------------
classifier.fit(input_fn =my_input_fn, steps=30)


SVM(params={'optimizer': <tensorflow.contrib.linear_optimizer.python.sdca_optimizer.SDCAOptimizer object at 0x7f7e1aaf3710>, 'head': <tensorflow.contrib.learn.python.learn.estimators.head._BinarySvmHead object at 0x7f7e1aaf3748>, 'feature_columns': [_RealValuedColumn(column_name='volatile_acidity', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='total_sulfur_dioxide', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='sulphates', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='alcohol', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)], 'weight_column_name': None, 'update_weights_hook': <tensorflow.contrib.learn.python.learn.estimators.linear._SdcaUpdateWeightsHook object at 0x7f7e1aaf3898>})

## Evaluation

Create a predict_fn() to supply data to make predictions.  
Then call classifier.predict() to create y_pred.

In [9]:
# Complete code below this comment  (Question #P6006)
# ----------------------------------
def predict_fn():
    columns = {
        feature_name: tf.constant(np.expand_dims(X_test[:,i], 1))
            for i,feature_name in enumerate(feature_names)
    }
    columns['example_id'] = tf.constant([str(i+1) for i in range(len(X_test))])
    return columns

Then call classifier.predict() to create **y_pred** as predictions.

**Hint**: See LinearSVM lab.

In [12]:
# Add code below this comment  (Question #P6007)
# ----------------------------------
y_pred= classifier.predict(input_fn = predict_fn)



Feed predictions **y_pred** along with ground truth **y_test** to confusion_matrix() to create a confusion matrix.

In [13]:
# Add code below this comment  (Question #P6008)
# ----------------------------------
y_pred = list(map(lambda i: i['classes'], y_pred))
confusion_matrix(y_test,y_pred)


array([[145,  44],
       [ 62, 149]])

# Save your notebook!